# Ruby データツールによるデータ解析

このノートブックでは、Daru などの Ruby のデータツールを利用したデータ解析を実演します。

Ruby には、機械学習や統計解析などを駆使した高度なデータサイエンスをするためのツールが揃っていないため、pycall.rb を使用して Python のデータツールによって補う必要があります。しかし、Ruby のデータツールだけでもフィルタリングや集計処理などは可能です。そして、データサイエンスのワークフローでは、多くの部分をそのような単純なデータ処理が支えています。本ノートブックを通して Ruby のデータツールの使い方を理解すれば、データサイエンスの多くの部分で Ruby を利用できるようになるでしょう。

## 本ノートブックで利用するデータツール

本ノートブックでは次のライブラリを利用します。

- [daru](https://github.com/SciRuby/daru): データフレーム機能を提供するライブラリ (Ruby 用の pandas のようなもの)
- [rbplotly](https://github.com/ash1day/rbplotly): [Plotly](https://plot.ly/) を利用してデータの可視化を行うライブラリ

これらのライブラリをまだインストールしていない場合は、次のコマンドを実行してインストールしてください。

```console
gem install daru rbplotly
```

## 本ノートブックで利用するデータ

本ノートブックでは、[Kaggle で提供されている Titanic の乗客データ](https://www.kaggle.com/c/titanic/data) を利用します。以下の手順にしたがってデータを取得してください。

### Kaggle へのユーザ登録

Kaggle で提供されているデータをダウンロードするには Kaggle へのユーザ登録とサインインが必要です。すでに Kaggle にサインイン済みであれば、次の節へ進んでデータをダウンロードしてください。

Kaggle へのユーザ登録とサインインを済ませていない場合は <https://www.kaggle.com/?login=true> にアクセスしてユーザ登録とサインインを実施してください。

### データセットのダウンロード

Kaggle へのユーザ登録が完了すると、下記の URL をクリックするだけでデータをダウンロードできます。

- [train.csv](https://www.kaggle.com/c/3136/download/train.csv)
- [test.csv](https://www.kaggle.com/c/3136/download/test.csv)

以下では、ノートブックファイルと同じディレクトリ内に上記2つのファイルがある前提で話を進めています。必要に応じて、ファイルのパスを変更して対応してください。

## Titanic データについて

(あとで書く)

## データのロード

daru を利用して `train.csv` ファイルをロードしデータフレームを作ります。まずは daru をロードします。

In [1]:
require 'daru'

true

続いて `Daru::DataFrame.from_csv` メソッドで `train.csv` ファイルを読み込みます。結果のデータフレームは変数 `df` に入れましょう。

In [2]:
df = Daru::DataFrame.from_csv('train.csv')
nil

セルの最後に `nil` を置いたのは、セルの実行結果を空にするためです。これを置かないと、変数 `df` への代入式の結果、つまり作成されたデータフレームオブジェクトがセルの実行結果になり、データフレーム全体が表示されてしまいます。

## データの確認

データの行数は `df.size` でわかります。データフレーム全体で何行あるか調べてみましょう。

In [3]:
df.size

891

行数がわかったので、次はデータフレームの先頭10行の内容を見てみましょう。

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S
5,6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C


## データの統計的な性質を調べる

`df.describe` メソッドを呼び出すことで、データフレーム無いの数値カラムを対象に統計的サマリを算出して表示できます。やってみましょう。

In [5]:
df.describe

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891,891,891,714,891,891,891
mean,446.0,0.3838383838383838,2.308641975308642,29.69911764705882,0.5230078563411896,0.38159371492704824,32.2042079685746
std,257.3538420152301,0.4865924542648585,0.8360712409770513,14.526497332334044,1.1027434322934275,0.8060572211299559,49.693428597180905
min,1,0,1,0.42,0,0,0
max,891,1,3,80,8,6,512.3292


この表の `Age` カラムを見ると、乗客の平均年齢は30歳で、最年少は0歳、最年長は80歳だったことが分かります。このような単純な統計サマリを見るだけでもデータの性質を理解する役に立つのですが、daru を使えば Ruby でもメソッド1発で統計サマリを確認できるのです。

`Survived` カラムの値と次の各カラムの関係を図示してみましょう。

- `Pclass`
- `Age`
- `SibSp`
- `Parch`
- `Fare`

まず、チャートを描くために使用する rbplotly ライブラリをロードします。

In [6]:
require 'rbplotly'

true

次に、`Plotly::Plot` オブジェクトの `#show` メソッドを利用して、チャートを描きます。

次のコードは、`PassengerId` 以外の数値カラムについて、それぞれのヒストグラムを格子状に並べて描くコードです。

In [7]:
def show_histograms(dataframe, names, layout: nil)
  layout ||= {}
  raise ArgumentError, "layout must be a Hash" unless layout.kind_of? Hash

  traces = names.map.with_index do |name, i|
    axis_index = i == 0 ? '' : i + 1
    {
      x: dataframe[name].to_a,
      name: name,
      type: :histogram,
      xaxis: "x#{axis_index}",
      yaxis: "y#{axis_index}",
      nbinsx: 20
    }
  end

  ncol = 2
  nrow = (names.length + 1) / ncol
  grid_width = 1.0 / ncol
  grid_height = 1.0 / nrow
  sep = 0.02

  trace_index = 0
  nrow.times do |row|
    ncol.times do |col|
      axis_index = trace_index == 0 ? '' : trace_index + 1
      layout[:"xaxis#{axis_index}"] = {
        domain: [col * grid_width + sep, (col + 1) * grid_width - sep],
        anchor: "y#{axis_index}"
      }
      layout[:"yaxis#{axis_index}"] = {
        domain: [row * grid_height + sep, (row + 1) * grid_height - sep],
        anchor: "x#{axis_index}"
      }
      trace_index += 1
    end
  end
  Plotly::Plot.new(data: traces, layout: layout).show
end

:show_histograms

In [8]:
num_column_names = df.numeric_vector_names - ['PassengerId']
show_histograms(df, num_column_names, layout: { height: 700 })

#<Plotly::Offline::HTML:0x000000000339c440 @id="380c4ebb-9069-48dd-a5ed-8e820f4aba4b", @data=[{:x=>[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], :name=>"Survived", :type=>:histogram, :xaxis=>"x", :yaxis=>"y", :nbinsx=>20}, {:x=>[3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 1, 3, 3, 3, 2, 3, 2, 3, 3, 2, 2, 3, 1, 3, 3, 3, 1, 3, 3, 1, 1, 3, 2, 1, 1, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 1, 1, 2, 3, 2, 3, 3, 1, 1, 3, 1, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 1, 2, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 1, 1, 2, 2, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 2, 1, 3, 2, 3, 2, 2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 3, 1, 3, 3, 3, 3, 2, 2, 3, 3, 2, 2, 2, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 1, 3, 1, 3, 1, 3, 3, 3, 1, 3, 3, 1, 2, 3, 3, 2, 3, 2, 3, 1, 3, 1, 3, 3, 2, 2, 3, 2, 1, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 2, 3, 2, 3, 1, 3, 2, 1, 2, 3, 2, 3, 3, 1, 3, 2, 3, 2, 3, 1, 3, 2, 3, 2, 3, 2, 2, 2, 2, 3, 3, 2, 3, 3, 1, 3, 2, 1, 2, 3, 3, 1, 3, 3, 3, 1, 1, 1, 2, 3, 3, 1, 1, 3, 2, 3, 3, 1, 1, 1, 3, 2, 1, 3, 1, 3, 2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 1, 1, 2, 3, 3, 1, 3, 1, 1, 1, 3, 3, 3, 2, 3, 1, 1, 1, 2, 1, 1, 1, 2, 3, 2, 3, 2, 2, 1, 1, 3, 3, 2, 2, 3, 1, 3, 2, 3, 1, 3, 1, 1, 3, 1, 3, 1, 1, 3, 1, 2, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 1, 3, 3, 3, 3, 1, 2, 3, 3, 3, 2, 3, 3, 3, 3, 1, 3, 3, 1, 1, 3, 3, 1, 3, 1, 3, 1, 3, 3, 1,

#<CZTop::Socket::PUB:0x35913b0 last_endpoint="tcp://127.0.0.1:57627">

同じものを、今度は男性の乗客について描いてみましょう。

In [9]:
df_male = df.filter(:row) {|r| r['Sex'] == 'male' }
show_histograms(df_male, num_column_names, layout: { height: 700 })

#<Plotly::Offline::HTML:0x0000000004ec12c0 @id="480265cd-7551-4295-a19f-7db30ec24403", @data=[{:x=>[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], :name=>"Survived", :type=>:histogram, :xaxis=>"x", :yaxis=>"y", :nbinsx=>20}, {:x=>[3, 3, 3, 1, 3, 3, 3, 3, 2, 2, 2, 1, 3, 1, 3, 1, 2, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 1, 3, 1, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 1, 2, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 2, 1, 2, 3, 2, 1, 3, 3, 3, 3, 3, 3, 2, 2, 1, 3, 1, 3, 2, 2, 3, 2, 2, 2, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 2, 3, 2, 1, 1, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 2, 3, 2, 2, 3, 2, 3, 3, 1, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3, 3, 1, 1, 2, 3, 1, 3, 3, 3, 1, 1, 2, 3, 3, 1, 3, 1, 2, 3, 3, 3, 3, 3, 1, 3, 3, 3, 2, 2, 3, 1, 3, 1, 3, 3, 3, 1, 2, 3, 2, 2, 3, 3, 3, 3, 1, 1, 3, 3, 1, 3, 1, 2, 2, 2, 2, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 3, 3, 1, 1, 3, 3, 3, 3, 2, 3, 3, 1, 3, 3, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 3, 3, 3, 1, 1, 1, 2, 3, 1, 1, 3, 3, 1, 3, 1, 3, 1, 2, 3, 3, 2, 1, 3, 3, 3, 1, 2, 3, 3, 3, 2, 3, 1, 1, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3, 1, 3, 2, 1, 1, 2, 3, 2, 1, 2, 3, 3, 1, 1, 3, 3, 2, 3, 3, 3, 3, 3, 2, 1, 3, 3, 3, 2, 1, 3, 2, 1, 3, 3, 3, 3, 2, 3, 3, 3, 1, 3, 1, 3, 1, 3, 3, 1, 3, 3, 3, 3, 2, 3, 1, 3, 3, 3, 1, 2, 3, 3, 1, 3, 1, 1, 3, 2, 3, 3, 3, 1, 3, 1, 3, 3, 3, 2, 3, 2, 1, 1, 3, 1, 3, 3, 2, 2, 3, 3, 1, 2, 2, 2, 3, 3, 1, 1, 3, 3, 2, 2, 3, 3, 3, 1, 3, 3, 1, 2, 3, 1, 3, 1, 3, 3, 2, 1, 3, 1, 1, 3, 2, 3, 3, 3, 3, 2, 2, 1, 3, 2, 3, 2, 2, 2, 3, 1, 3, 3, 1, 1, 3, 3, 1, 3, 1, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 2, 1, 3, 2, 3, 2, 1, 3, 3, 3, 1, 2, 3, 3, 2, 3, 1, 2, 3, 3, 3, 1, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 1, 3, 3, 2, 2, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3, 2, 1, 3], :name=>"Pclass", :type=>:histogram, :xaxis=>"x2", :yaxis=>"y2", :nbinsx=>20}, {:x=>[22, 35, nil, 54, 2, 20, 39, 2, nil, 35, 34, 28, nil, 19, nil, 40, 66, 28, 42, nil, 21, nil, nil, nil, nil, 7, 21, 65, nil, 28.5, 11, 22, 45, 4, nil, nil, 19, 26, 32, 21, 26, 32, 25, nil, nil, 0.83, 22, 29, 28, 16, nil, 24, 29, 20, 46, 26, 59, nil, 71, 23, 34, nil, 21,

#<CZTop::Socket::PUB:0x35913b0 last_endpoint="tcp://127.0.0.1:57627">

さらに、女性の乗客についても描いてみます。

In [10]:
df_female = df.filter(:row) {|r| r['Sex'] == 'female' }
show_histograms(df_female, num_column_names, layout: { height: 700 })

#<Plotly::Offline::HTML:0x0000000004d872b0 @id="c37b8900-44da-4c23-94eb-a557d2558ebc", @data=[{:x=>[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0], :name=>"Survived", :type=>:histogram, :xaxis=>"x", :yaxis=>"y", :nbinsx=>20}, {:x=>[1, 3, 1, 3, 2, 3, 1, 3, 2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 2, 2, 3, 3, 3, 1, 2, 2, 2, 1, 2, 3, 3, 3, 3, 2, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 1, 3, 3, 3, 3, 1, 3, 2, 1, 3, 3, 1, 3, 3, 3, 2, 3, 1, 1, 3, 2, 3, 3, 2, 1, 3, 1, 3, 1, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 1, 1, 1, 2, 3, 1, 1, 2, 3, 1, 3, 3, 3, 1, 1, 3, 1, 1, 3, 2, 1, 1, 1, 1, 1, 2, 3, 2, 1, 1, 2, 2, 1, 2, 3, 1, 3, 1, 1, 1, 2, 2, 3, 1, 2, 3, 3, 3, 1, 3, 3, 1, 3, 1, 3, 1, 3, 1, 2, 2, 1, 3, 3, 2, 3, 3, 3, 1, 3, 2, 2, 3, 3, 2, 2, 3, 2, 1, 3, 2, 2, 2, 2, 3, 1, 2, 3, 2, 2, 3, 3, 3, 3, 1, 1, 1, 3, 3, 3, 1, 2, 1, 2, 2, 1, 1, 2, 2, 3, 3, 2, 1, 1, 1, 3, 3, 2, 3, 1, 1, 3, 3, 3, 1, 3, 2, 1, 3, 2, 1, 1, 1, 3, 2, 2, 2, 1, 3, 3, 2, 3, 2, 1, 3, 2, 3, 1, 3, 3, 3, 2, 3, 3, 3, 1, 2, 3, 3, 3, 1, 3, 3, 1, 3, 2, 1, 1, 1, 2, 2, 2, 3, 3, 1, 3, 1, 2, 2, 2, 1, 1, 1, 3, 2, 2, 3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 1, 3, 3, 1, 3, 1, 3, 1, 1, 1, 3, 1, 2, 3, 1, 3, 1, 3, 2, 2, 1, 2, 3, 1, 2, 3, 3, 1, 3], :name=>"Pclass", :type=>:histogram, :xaxis=>"x2", :yaxis=>"y2", :nbinsx=>20}, {:x=>[38, 26, 35, 27, 14, 4, 58, 14, 55, 31, nil, 15, 8, 38, nil, nil, nil, 18, 14, 40, 27, 3, 19, nil, 18, 49, 29, 21, 5, 38, 29, 17, 16, 30, nil, 17, 33, 23, 34, 28, 21, nil, 14.5, 20, 17, 2, 32.5, nil, 47, 29, 19, nil, 22, 24, 9, 22, 16, 40, nil, 45, 1, 50, nil, 4, nil, 32, 19, 44, 58, nil, 24, 2, 16, 35, 31, 27, 32, nil, 35, 5, nil, 8, nil, nil, 25, 24, 29, 41, 29, nil, 30, 35, 50, nil, 58, 35, 41, nil, 63, 45, 35, 22, 26, 19, 24, 2, 50, nil, nil, nil, 17, 30, 24, 18, 26, 26, 24, 31, 40, 30, 22, 36, 36, 31, 16, nil, nil, 41, 24, 24, 40, nil, 22, 38, nil, nil, 45, 60, nil, nil, 24, 3, nil, 22, 42, 1, 35, 36, 17, 23, 24, 31, 28, 21, 20, nil, 33, nil, 34, 18, 10, 28, 28, 19, nil, 42, 14, 21, 24, 45, 28, 13, 5, nil, 50, 0.75, 33, 23, 22, 2, 63, nil, 35, 54, 25, 21, nil, 37, 16, 33, 54, 34, 36, 30, 44, 50, 2, nil, 30, 7, 30, 22, 36, 9, 11, 19, 22, 48, 39, 36, nil, 29, 53, nil, 34, 39, nil, 25, 39, 18, 52, nil, nil, 24, 22, 40, 39, nil, 24, 26, 4, 21, 9, 28, 41, 24, 2, 0.75, 23, 18, nil, 18, 32, nil, 40, 18, 43, nil, 15, 4, nil, 18, 18, 45, 22, 24, 38, 27, 6, 30, nil, 25, 29, 48, 21, 30, 4, 48, 33, 36, 51, 30.5, 57, 54, 5, 43, 13, 17, 18, nil, 49, 31, 30, 31, 18, 33, 6, 23, 52, 27, 62, 15, 39, 30, nil, 9, 16, 44, 18, 45, 24, 48, nil, 42, 27, 47, 28, 15, 56, 25, 22, 39, 19, nil], :name=>"Age", :type=>:histogram, :xaxis=>"x3", :yaxis=>"y3", :nbinsx=>20}, {:x=>[1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 3, 1, 0, 1, 0, 2, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 4, 5, 0, 0, 0, 3, 3, 0, 0, 0, 1, 1, 1, 0, 4, 0, 1, 1, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 1, 1, 0, 8, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 1, 4, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 2, 1, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 3, 1, 0,

#<CZTop::Socket::PUB:0x35913b0 last_endpoint="tcp://127.0.0.1:57627">

男性と女性の違いを分かりやすく可視化するため、各ヒストグラムについて男女別に色を変えて同時にプロットしてみましょう。

In [11]:
def show_histograms_overlay_about_sex(dataframe, names, layout: nil)
  layout ||= {}
  raise ArgumentError, "layout must be a Hash" unless layout.kind_of? Hash

  group_by_sex = dataframe.group_by('Sex')
  male = group_by_sex.get_group(['male'])
  female = group_by_sex.get_group(['female'])

  traces = names.map.with_index { |name, i|
    axis_index = i == 0 ? '' : i + 1
    [
      {
        x: male[name].to_a,
        name: "#{name} (male)",
        type: :histogram,
        xaxis: "x#{axis_index}",
        yaxis: "y#{axis_index}",
        nbinsx: 20
      },
      {
        x: female[name].to_a,
        name: "#{name} (female)",
        type: :histogram,
        xaxis: "x#{axis_index}",
        yaxis: "y#{axis_index}",
        nbinsx: 20
      }
    ]
  }.flatten

  ncol = 2
  nrow = (names.length + 1) / ncol
  grid_width = 1.0 / ncol
  grid_height = 1.0 / nrow
  sep = 0.02

  trace_index = 0
  nrow.times do |row|
    ncol.times do |col|
      axis_index = trace_index == 0 ? '' : trace_index + 1
      layout[:"xaxis#{axis_index}"] = {
        domain: [col * grid_width + sep, (col + 1) * grid_width - sep],
        anchor: "y#{axis_index}"
      }
      layout[:"yaxis#{axis_index}"] = {
        domain: [row * grid_height + sep, (row + 1) * grid_height - sep],
        anchor: "x#{axis_index}"
      }
      trace_index += 1
    end
  end
  Plotly::Plot.new(data: traces, layout: layout).show
end

:show_histograms_overlay_about_sex

In [12]:
show_histograms_overlay_about_sex(df, num_column_names, layout: { height: 700 })

#<Plotly::Offline::HTML:0x0000000004caf770 @id="69d6ba94-42cb-4545-a89e-200b5b5c8020", @data=[{:x=>[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], :name=>"Survived (male)", :type=>:histogram, :xaxis=>"x", :yaxis=>"y", :nbinsx=>20}, {:x=>[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0], :name=>"Survived (female)", :type=>:histogram, :xaxis=>"x", :yaxis=>"y", :nbinsx=>20}, {:x=>[3, 3, 3, 1, 3, 3, 3, 3, 2, 2, 2, 1, 3, 1, 3, 1, 2, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 1, 3, 1, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 1, 2, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 2, 1, 2, 3, 2, 1, 3, 3, 3, 3, 3, 3, 2, 2, 1, 3, 1, 3, 2, 2, 3, 2, 2, 2, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 2, 3, 2, 1, 1, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 2, 3, 2, 2, 3, 2, 3, 3, 1, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3, 3, 1, 1, 2, 3, 1, 3, 3, 3, 1, 1, 2, 3, 3, 1, 3, 1, 2, 3, 3, 3, 3, 3, 1, 3, 3, 3, 2, 2, 3, 1, 3, 1, 3, 3, 3, 1, 2, 3, 2, 2, 3, 3, 3, 3, 1, 1, 3, 3, 1, 3, 1, 2, 2, 2, 2, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 3, 3, 1, 1, 3, 3, 3, 3, 2, 3, 3, 1, 3, 3, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 3, 3, 3, 1, 1, 1, 2, 3, 1, 1, 3, 3, 1, 3, 1, 3, 1, 2, 3, 3, 2, 1, 3, 3, 3, 1, 2, 3, 3, 3, 2, 3, 1, 1, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3, 1, 3, 2, 1, 1, 2, 3, 2, 1, 2, 3, 

#<CZTop::Socket::PUB:0x35913b0 last_endpoint="tcp://127.0.0.1:57627">

こうして、男女別のヒストグラムを同時にプロットすると、例えば生存者は女性の方が多く、死亡者は圧倒的に男性の方が大きかったことなど、男女による傾向の違いがはっきりと分かりますね。

このように、Ruby のためのデータツールを使うだけでも、データの性質を調べることが可能です。

## Ruby のデータツールが抱える課題

これまで、Ruby のデータツールだけでデータ解析を実践してきました。使用したものは daru と rbplotly だけですが、この2つについて次のような課題があることが分かります。

### daru が抱える課題

daru が抱える問題として、データのフィルタリングが pandas と比べて面倒であることが分かります。特にデータフレームをフィルタリングする次のようなコードです。

```ruby
df_male = df.filter(:row) {|r| r['Sex'] == 'male' }
```

これは、`Sex` カラムの値が `'male'` である行のみを抽出した新しいデータフレームを作成します。同じことを pandas では次のように記述できます。

```python
df_male = df[df['Sex'] == 'male']
```

pandas の記法の方が daru よりも記述がシンプルであり、さらにこの真偽値ベクトルを用いた行フィルタリングはデータサイエンスにおいては一般的な記法なので、慣れると一瞥して何をやろうとしているのか分かります。一方、daru のブロック渡し記法は、Ruby における一般的なイテレータ記法であるため Ruby プログラマには親しみのあるパターンですが、その反面ほかのイテレータ記法と見た目が同じなので見分けにくい問題があります。

daru も真偽値ベクトルによる行フィルタリングに対応させたいですね。

### rbplotly が抱える課題

rbplotly は、`Plotly::Plot` オブジェクトに渡したデータをそのまま JSON に変換して Plotly の可視化システムに渡します。そのため、本ノートブックがやっているように、適切にデータとレイアウトを計算して作ってあげれば、Plotly が対応している全ての可視化方法を Ruby から利用できます。

しかしながら、Plotly の Python API が提供している figure factory などの便利機能はありません。そのような便利機能を実装していくことが rbplotly の今後の課題です。

## まとめ

本ノートブックでは、Titanic データを用いて、Ruby のデータツールだけでも可能なデータ解析を実践しました。そして、データ解析の実践を通して判明した daru と rbplotly の課題について述べました。